In [ ]:
from main import main, context_length, prediction_length
from vierlinden.data.loader import VierlindenDataProcessor
from vierlinden.config import model_output_path

target = "Entleerung_RüB"
model_filepath = model_output_path + "/NHiTS"  + target + "_model.pt"

dp = VierlindenDataProcessor()
df = dp.load_processed_data()
df = dp.prepare_for_target(df, target)
dp.plot_target_col(target)

## Training

In [ ]:
training_wrapper, test_df = main(target)
training_wrapper.save_trained_model(model_filepath)

print(training_wrapper.metrics_callback.metrics["train_loss"])
print(training_wrapper.metrics_callback.metrics["val_loss"])

training_wrapper.plot_training_result()

## Testing

In [ ]:
from vierlinden.model.model import NHiTSTrainingWrapper, NHiTSPredictionWrapper

model = NHiTSTrainingWrapper.load_trained_model(model_filepath)
prediction_wrapper = NHiTSPredictionWrapper(model, context_length=context_length, prediction_length=prediction_length, target_col=target)

if test_df is None:
    df = dp.load_processed_data()
    df = dp.prepare_for_target(df, target)
    training_df, test_df = dp.split_data(df)
    
prediction_df = prediction_wrapper.predict(test_df)
mase = prediction_wrapper.calculate_mase_loss()

print("MASE = ", mase)

In [ ]:
prediction_df.iloc[110:120]

In [ ]:
prediction_wrapper.plot_forecast_for_all(forecast_step_ahead=1, plot_forecast_distribution=True)

In [ ]:
prediction_wrapper.plot_forecast_per_time_idx(115)
prediction_wrapper.plot_forecast_per_time_idx(120)